In [1]:
!pip install mlserver
!pip install boto3

  Using cached mlserver-1.3.5-py3-none-any.whl (113 kB)
  Using cached aiokafka-0.9.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached orjson-3.9.10-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (138 kB)
  Using cached starlette_exporter-0.17.1-py3-none-any.whl (14 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Using cached tritonclient-2.40.0-py3-none-manylinux1_x86_64.whl (12.4 MB)
  Using cached aiofiles-23.2.1-py3-none-any.whl (15 kB)
  Using cached uvloop-0.19.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.0 MB)
  Using cached uvicorn-0.24.0.post1-py3-none-any.whl (59 kB)
  Using cached grpcio-1.59.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.3 MB)
  Using cached fastapi-0.89.1-py3-none-any.whl (55 kB)
  Using cached py_grpc_prometheus-0.7.0-py3-none-any.whl (12 kB)
  Using cached pydantic-1.10.13-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
  Using cached starlette-0

In [47]:
import pandas as pd
import requests
import json
from mlserver.codecs import PandasCodec

In [49]:
dataframe = pd.DataFrame({
    'First Name': ["Joanne", "Michael"], 
    'Age': [34, 22], 
    'Deal': [
        {'arg1': 0, 'arg2': [1, 2]}, 
        {'arg1': 10, 'arg2': [10, 20]}
    ]
})

inference_request = PandasCodec.encode_request(dataframe)

In [50]:
inference_request

InferenceRequest(id=None, parameters=Parameters(content_type='pd', headers=None), inputs=[RequestInput(name='First Name', shape=[2, 1], datatype='BYTES', parameters=Parameters(content_type='str', headers=None), data=TensorData(__root__=[b'Joanne', b'Michael'])), RequestInput(name='Age', shape=[2, 1], datatype='INT64', parameters=None, data=TensorData(__root__=[34, 22])), RequestInput(name='Deal', shape=[2, 1], datatype='BYTES', parameters=None, data=TensorData(__root__=[{'arg1': 0, 'arg2': [1, 2]}, {'arg1': 10, 'arg2': [10, 20]}]))], outputs=None)

In [51]:
raw_request = inference_request.json()
raw_request = eval(raw_request)
raw_request

{'parameters': {'content_type': 'pd'},
 'inputs': [{'name': 'First Name',
   'shape': [2, 1],
   'datatype': 'BYTES',
   'parameters': {'content_type': 'str'},
   'data': ['Joanne', 'Michael']},
  {'name': 'Age', 'shape': [2, 1], 'datatype': 'INT64', 'data': [34, 22]},
  {'name': 'Deal',
   'shape': [2, 1],
   'datatype': 'BYTES',
   'data': [{'arg1': 0, 'arg2': [1, 2]}, {'arg1': 10, 'arg2': [10, 20]}]}]}

In [52]:
ext = "http://mlserver-120-0.seldon-mesh.svc:9000/v2/models/pandas-reader-v6_1/infer"
response = requests.post(ext, json=raw_request)

In [53]:
print(json.dumps(response.json(), indent=2))

{
  "model_name": "pandas-reader-v6_1",
  "model_version": "1",
  "id": "873556a8-6d4c-4880-bee6-a05344af5ad4",
  "parameters": {
    "content_type": null,
    "headers": null
  },
  "outputs": [
    {
      "name": "First Name",
      "shape": [
        2,
        1
      ],
      "datatype": "BYTES",
      "parameters": null,
      "data": [
        "Joanne",
        "Michael"
      ]
    },
    {
      "name": "Age",
      "shape": [
        2,
        1
      ],
      "datatype": "INT64",
      "parameters": null,
      "data": [
        34,
        22
      ]
    },
    {
      "name": "Deal",
      "shape": [
        2,
        1
      ],
      "datatype": "BYTES",
      "parameters": null,
      "data": [
        {
          "arg1": 0,
          "arg2": [
            1,
            2
          ]
        },
        {
          "arg1": 10,
          "arg2": [
            10,
            20
          ]
        }
      ]
    },
    {
      "name": "status",
      "shape": [
        

Загрузка файла модели в s3

In [20]:
import os
os.environ['KUBERNETES_S3_BUCKET']

'app-cor'

In [23]:
import boto3

bucket_name = os.environ.get("S3_BUCKET")
project_s3_client = boto3.client("s3", 
            endpoint_url=os.environ.get("FEAST_S3_ENDPOINT_URL"),
            aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
            aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY")
)


In [24]:
r = project_s3_client.upload_file('pandas_reader/model.py', bucket_name, 'pandas_reader/model.py')

In [37]:
r

app-cor/pandas_reader/model.py